In [ ]:
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPool1D
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import Sequential
import tensorflow as tf
import csv
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import os
import hickle as hkl
import copy
from scipy.spatial import distance_matrix
import sys
import random
import math

In [ ]:
# calculating dataset size weight per client
local_coeffs = {}
for i in range(0,clientCount):
    local_coeffs[i] = np.float32(len(clientLabelTrain[i])) / np.float32(len(centralTrainLabel))

In [ ]:
# initializing client model
local_nets = {}
local_histories = {}

for i in range(0,clientCount):
    local_nets[i] = create_keras_model()

In [ ]:
# initialization of client distance (for distance measurement)
clientEuclidDistMean = {}
clientEuclidDistStd = {}
for i in range(clientCount):
    clientEuclidDistMean[i] = np.zeros(communicationRound)
    clientEuclidDistStd[i] = np.zeros(communicationRound)

# Federated learning training
for roundNum in range(0, communicationRound):
    start_time = time.time()
    trainAcc = []
    trainLoss = []

    testAcc = []
    testLoss = []

    clientTrainAcc = []
    clientTrainLoss = []

    clientTestAcc = []
    clientTestLoss = []

    local_weights = {}

    if (asyncTest):
        if (roundNum in roundEnd):
            for i in range(clientDeleteCount):
                if (len(trainPool) != 0):
                    selection = random.choice(list(enumerate(trainPool)))
                    del trainPool[selection[0]]
                    idlePool.append(selection[1])
            for i in range(clientAddCount):
                if (len(idlePool) != 0):
                    selection = random.choice(list(enumerate(idlePool)))
                    del idlePool[selection[0]]
                    trainPool.append(selection[1])

        participantDataInstance = []
        for index, i in enumerate(trainPool):
            participantDataInstance.append(clientLabelTrain[i])
        participantDataInstance = (np.hstack((participantDataInstance)))
        local_coeffs = {}
        for index, i in enumerate(trainPool):
            local_coeffs[i] = np.float32(len(clientLabelTrain[i])) / np.float32(len(participantDataInstance))
    for index, i in enumerate(trainPool):
        print("Status: Round #" + str(roundNum) + " Client #" + str(i))

        if (algorithm == "FEDPER"):
            local_nets[i].load_weights(filepath + 'serverWeights.h5', by_name=True)
        else:
            local_nets[i].load_weights(filepath + 'serverWeights.h5', by_name=False)
        if (optimizer == "SGD"):
            local_nets[i].compile(optimizer=SGD(learning_rate=learningRate), loss='sparse_categorical_crossentropy',
                                  metrics=['acc'])
        else:
            local_nets[i].compile(optimizer=Adam(learning_rate=learningRate), loss='sparse_categorical_crossentropy',
                                  metrics=['acc'])
        local_histories[i] = local_nets[i].fit(clientDataTrain[i], clientLabelTrain[i],
                                               class_weight=local_class_weights[i], epochs=localEpoch,
                                               verbose=showTrainVerbose)

        local_weights[i] = local_nets[i].get_weights()
        trainAcc.append(local_histories[i].history['acc'])
        trainLoss.append(local_histories[i].history['loss'])

        #         testing againts their own testset
        testModelMetrics = local_nets[i].evaluate(clientDataTest[i], clientLabelTest[i], verbose=showTrainVerbose)
        testAcc.append(testModelMetrics[1])
        testLoss.append(testModelMetrics[0])

        if (ClientAllTest == True):
            #           testing againts their all testset
            clientTrainModelMetrics = local_nets[i].evaluate(centralTrainData, centralTrainLabel,
                                                             verbose=showTrainVerbose)
            clientTrainAcc.append(clientTrainModelMetrics[1])
            clientTrainLoss.append(clientTrainModelMetrics[0])

            clientTestModelMetrics = local_nets[i].evaluate(centralTestData, centralTestLabel, verbose=showTrainVerbose)
            clientTestAcc.append(clientTestModelMetrics[1])
            clientTestLoss.append(clientTestModelMetrics[0])

        for j in range(0, len(local_weights[i])):
            local_weights[i][j] = local_weights[i][j] * local_coeffs[i]

    if (euclid):
        meanServerClient = []
        stdServerClient = []
        serverShape = np.asarray(computeWeights(serverModel.get_weights()))
        localMeanClientLayer = []
        localStdClientLayer = []
        for index, clientIndex in enumerate(trainPool):
            localMeanServerClient = []
            localStdServerClient = []

            localShape = np.asarray(computeWeights(local_nets[clientIndex].get_weights()))
            if (algorithm != 'FEDPER'):
                for i in range(serverShape.shape[0]):
                    newLayerDist = np.sqrt((serverShape[i] - localShape[i]) ** 2)
                    localMeanServerClient.append(np.mean(newLayerDist))
                    localStdServerClient.append(np.std(newLayerDist))
            else:
                newLayerDist = np.sqrt((serverShape[0] - localShape[0]) ** 2)
                localMeanServerClient.append(np.mean(newLayerDist))
                localStdServerClient.append(np.std(newLayerDist))

            localMeanClientLayer.append(localMeanServerClient)
            localStdClientLayer.append(localStdServerClient)
            meanServerClient.append(np.mean(localMeanServerClient))
            stdServerClient.append(np.mean(localStdServerClient))
            clientEuclidDistMean[clientIndex][roundNum] = np.mean(localMeanServerClient)
            clientEuclidDistStd[clientIndex][roundNum] = np.mean(localStdServerClient)

        #         15 clients
        meanHistoryDist.append(np.asarray(meanServerClient))
        stdHistoryDist.append(np.asarray(stdServerClient))

        #         per layer distance
        meanRoundLayerHistory.append(np.mean(localMeanClientLayer, axis=0))
        stdRoundLayerHistory.append(np.mean(localStdClientLayer, axis=0))

        #         all layer distance
        meanRoundGeneralLayerHistory.append(np.mean(localMeanClientLayer))
        stdRoundGeneralLayerHistory.append(np.mean(localStdClientLayer))

    trainAccHistory.append(np.mean(trainAcc))
    stdTrainAccHistory.append(np.std(trainAcc))
    trainLossHistory.append(np.mean(trainLoss))
    stdTrainLossHistory.append(np.std(trainLoss))

    meanTestAcc = np.mean(testAcc)

    testAccHistory.append(meanTestAcc)
    stdTestAccHistory.append(np.std(testAcc))
    testLossHistory.append(np.mean(testLoss))
    stdTestLossHistory.append(np.std(testLoss))

    if (meanTestAcc > currentAccuracy):
        for index, net in enumerate(local_nets):
            best_local_nets[index] = copy.copy(local_nets[index])
        currentAccuracy = meanTestAcc
        bestModelRound = roundNum + 1

    if (ClientAllTest == True):
        clientTrainLossHistory.append(np.mean(clientTrainLoss))
        clientTrainAccHistory.append(np.mean(clientTrainAcc))
        clientTestLossHistory.append(np.mean(clientTestLoss))
        clientTestAccHistory.append(np.mean(clientTestAcc))

        clientStdTrainLossHistory.append(np.std(clientTrainLoss))
        clientStdTrainAccHistory.append(np.std(clientTrainAcc))
        clientStdTestLossHistory.append(np.std(clientTestLoss))
        clientStdTestAccHistory.append(np.std(clientTestAcc))

    # return weights to server and sum all the model weights
    tf.keras.backend.clear_session()

    weights = []
    for i in local_weights:
        weights.append(local_weights[i])
    new_weights = list()
    for weights_list_tuple in zip(*weights):
        new_weights.append(np.asarray(
            [np.array(weights_).sum(axis=0)
             for weights_ in zip(*weights_list_tuple)]))
    serverModel.set_weights(np.asarray(new_weights))
    del new_weights
    del weights
    del weights_list_tuple

    #    FedDist main implemenation begins here

    for layer in range(len(layerType) - 1):
        if (layer != len(layerType) - 1):
            distanceMatrix = []
            allClientWeight = {}
            serverWeights = computeWeights(serverModel.get_weights())

            for i, clientIndex in enumerate(trainPool):
                clientWeights = computeWeights(local_nets[clientIndex].get_weights())
                allClientWeight[clientIndex] = clientWeights
                clientDistance = []
                for k in range(serverWeights[layer].shape[0]):
                    clientDistance.append(distance.euclidean(clientWeights[layer][k], serverWeights[layer][k]))
                distanceMatrix.append(clientDistance)

                del clientWeights
                del clientDistance

            distanceMatrix = np.asarray(distanceMatrix)
            means = np.mean(distanceMatrix, axis=0)
            stds = np.std(distanceMatrix, axis=0)
            stdThreshold = stdCount + np.floor(roundNum / 5) * 0.25
            threshHold = means + (stdThreshold * stds)

            nextLayerOutWeights = (allClientWeight[0][layer + 1].shape[1] - 1) / allClientWeight[0][layer].shape[0]
            newUnit = []
            outerUnit = []
            for i, clientIndex in enumerate(trainPool):
                for k in range(serverWeights[layer].shape[0]):
                    if (distanceMatrix[clientIndex][k] > threshHold[k]):
                        newUnit.append(allClientWeight[clientIndex][layer][k])
                        nextLayerIndexStart = int(nextLayerOutWeights * k)
                        nextLayerIndexEnd = int(nextLayerIndexStart + nextLayerOutWeights)
                        outerUnit.append(
                            allClientWeight[clientIndex][layer + 1][:, nextLayerIndexStart:nextLayerIndexEnd])
            del allClientWeight
            del nextLayerOutWeights
            if (len(newUnit) == 0):
                print("No new unit")
                continue
            else:
                print("New units added :" + str(len(newUnit)) + " on layer : " + str(layer))
            serverWeights[layer] = np.vstack((serverWeights[layer], newUnit))
            outwardsUnit = np.hstack(outerUnit)
            bias = serverWeights[layer + 1][:, serverWeights[layer + 1].shape[1] - 1:]
            outwardsUnit = np.hstack(
                (serverWeights[layer + 1][:, :serverWeights[layer + 1].shape[1] - 1], outwardsUnit))
            outwardsUnit = np.hstack((outwardsUnit, bias))
            serverWeights[layer + 1] = outwardsUnit
            newWeights = []
            for layerIndex in range(len(serverWeights)):
                sizeWithoutBias = serverWeights[layerIndex].shape[1] - 1
                if (layerType[layerIndex] == 0):
                    layerWeight = serverWeights[layerIndex][:, :sizeWithoutBias].T.reshape(16, 6, -1)
                else:
                    layerWeight = serverWeights[layerIndex][:, :sizeWithoutBias].T
                layerBias = serverWeights[layerIndex][:, sizeWithoutBias:].ravel()
                newWeights.append(layerWeight)
                newWeights.append(layerBias)

            del serverModel
            tf.keras.backend.clear_session()
            serverModel = createCNN(filter_count=newWeights[0].shape[2], dense_unit=newWeights[2].shape[1])
            serverModel.set_weights(newWeights)
            del serverWeights

            for i in range(layer + 1):
                serverModel.layers[layerMap[i]].trainable = False
            local_nets = {}
            local_weights = {}

            for i, index in enumerate(trainPool):
                tf.keras.backend.clear_session()
                local_nets[index] = createCNN(filter_count=newWeights[0].shape[2], dense_unit=newWeights[2].shape[1])
                local_nets[index].set_weights(newWeights)
                for i in range(layer + 1):
                    local_nets[index].layers[layerMap[i]].trainable = False
                if (optimizer == "SGD"):
                    local_nets[index].compile(optimizer=SGD(learning_rate=learningRate),
                                              loss='sparse_categorical_crossentropy', metrics=['acc'])
                else:
                    local_nets[index].compile(optimizer=Adam(learning_rate=learningRate),
                                              loss='sparse_categorical_crossentropy', metrics=['acc'])

                local_nets[index].fit(clientDataTrain[index], clientLabelTrain[index],
                                      class_weight=local_class_weights[index], epochs=localEpoch,
                                      verbose=showTrainVerbose)

                local_weights[index] = local_nets[index].get_weights()

                for j in range(0, len(local_weights[index])):
                    local_weights[index][j] = local_weights[index][j] * local_coeffs[index]

            weights = []
            for i in local_weights:
                weights.append(local_weights[i])
            new_weights = list()
            for weights_list_tuple in zip(*weights):
                new_weights.append(np.asarray(
                    [np.array(weights_).sum(axis=0)
                     for weights_ in zip(*weights_list_tuple)]))
            serverModel.set_weights(np.asarray(new_weights))
            del weights
            del weights_list_tuple
            del new_weights

            for i in range(layer + 1):
                serverModel.layers[layerMap[i]].trainable = True
                for index, clientId in enumerate(trainPool):
                    local_nets[clientId].layers[layerMap[i]].trainable = True
            tf.keras.backend.clear_session()
    serverModel.summary()

    #     Main FedDist implementations end here====================
    serverModel.save_weights(filepath + 'serverWeights.h5')
    if (algorithm != 'FEDPER'):
        if (optimizer == "SGD"):
            serverModel.compile(optimizer=SGD(learning_rate=learningRate), loss='sparse_categorical_crossentropy',
                                metrics=['acc'])
        else:
            serverModel.compile(optimizer=Adam(learning_rate=learningRate), loss='sparse_categorical_crossentropy',
                                metrics=['acc'])
        serverTrainMetrics = serverModel.evaluate(centralTrainData, centralTrainLabel, verbose=showTrainVerbose)
        serverTrainLossHistory.append(serverTrainMetrics[0])
        serverTrainAccHistory.append(serverTrainMetrics[1])

        serverTestMetrics = serverModel.evaluate(centralTestData, centralTestLabel, verbose=showTrainVerbose)
        serverTestLossHistory.append(serverTestMetrics[0])
        serverTestAccHistory.append(serverTestMetrics[1])
        if (serverTestMetrics[1] > serverCurrentAccuracy):
            serverCurrentAccuracy = serverTestMetrics[1]
            serverbestModelRound = roundNum + 1
            bestServerModel = copy.copy(serverModel)
        del serverTestMetrics
        del serverTrainMetrics
    tf.keras.backend.clear_session()
endTime = time.time() - start_time